In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## General Layout Style 
### Subheader in h3 format and notes in paragraph 

In [ ]:
#basic import
import numpy as np
import pandas as pd

#visualizations
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#machine learning
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Basic Data manupulation
### Goal
* Look into the data 
* Identify shape, structure
* Get a general overview 
* Run some autovisualizations


In [ ]:
train = pd.read_csv("../input/titanic/train.csv")


In [ ]:
test = pd.read_csv("../input/titanic/test.csv")

In [ ]:
test_ =test.copy()

In [ ]:
train.info()

* we have 2 float columns, 5 integer columns, 5 object columns
* Traget variable is survived and has zero null value.
* Age , cabin and Embarked has null values with them.

In [ ]:
test.info()

* cabin, age and far is missing some values here.
* test.csv is used to test our hypothesis model. 

### First Rows of data

In [ ]:
train.head()

In [ ]:
test.head()

### Some summary statistics of the data 

In [ ]:
train.describe()

Basic Understanding
* training data contains people where more people dies than survives.It cn be understood from survived.mean value
* Fare is not uniformly or normally distributed and skewed towards right as we have mean around 32 and max around 512.
* Average age of people travelling in titanic is around 29 
* Most of the people travels alone.(obvious from real world senario)

wow all these with simple train.describe()

In [ ]:
#run some auto visualization tool to see graphics of data no need to understand now
!pip install git+git://github.com/AutoViML/AutoViz.git
!pip install xlrd

### Autoviz

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()
dftc = AV.AutoViz(
    filename='', 
    sep='' , 
    depVar='Survived', 
    dfte=train, 
    header=0, 
    verbose=1, 
    lowess=False, 
    chart_format='png', 
    max_rows_analyzed=300000,
    max_cols_analyzed=30
)

# Pandas Profilling

In [ ]:
from pandas_profiling import ProfileReport
report = ProfileReport(train)
report

* This visuals will help us get understand the data more accuately 
* We will create more visualization to help understand the data 
* After understanding of the data we will go through a pipeline process to get cleaned data
* atlast applying a machine learning model to validate our skills.

## Handling Missing Value 

In [ ]:
median = train['Age'].median()
train['Age'].fillna(value=train['Age'].median(), inplace=True)

In [ ]:
train["Embarked"].fillna(value="Q", inplace=True)

# Data Visualization(EDA)

### Survived data field
Analysis of survived data field

In [ ]:
survived = train["Survived"]

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(4,4),dpi=80)
sns.countplot(survived)

In [ ]:
did_not_survived=round(len(survived[survived==0])/len(survived),2)*100
print(f"Their are {did_not_survived}% of people who lost there life in the mishap")
print("We get this information from train dataset")

* Data set is partially imbalanced.

# Other columns interaction with Survived

### Corelation plot

In [ ]:
corr = train.corr()
plt.figure(figsize=(8,4))
sns.heatmap(corr,annot=True,cmap="cubehelix")

* survived has corelation with fare and pclass. Where pclass and fare is also corelated.

* pclass is also corelated with age. does this make any sense? may be with age you earn money and you will get to buy good class tickits. make sense. 

### Fare and Survived

In [ ]:
plt.figure(figsize=(8,4))
sns.catplot(x="Survived",y="Fare",data=train,kind="swarm")


* Most of the peoples fare remains below 100
* With incresing number of fare the survival rate also increase 
* For a Fare of greater than 500 you will sure shot survived in Titanic.

In [ ]:
plt.figure(figsize=(10,6))
sns.catplot(x="Survived",y="Fare",data=train,kind="swarm",hue="Sex")

* Interesting Fact most of the dead people are male so gender is crucial parameter for detection of survival.
* A high percent of women survived this mishap regardless of their wealth status.(Assumption: Wealth of a person is proportional to the fare of ticket) 

In [ ]:
sns.catplot(x="Survived",y="Fare",data=train,kind="swarm",hue="Pclass",palette="rocket")

* One simple understanding if you are poor you will die.
* Female has more advantage than man over living

In [ ]:
fig = px.box(train,y="Fare",x="Pclass",points="all",color="Survived")
fig.show()

* Apart from class 1 the data remains concentrated into other classes
* Class 1 and 2 has higher median of survival with fare.

In [ ]:
fig = px.box(train,y="Fare",x="Survived",points="all",color="Survived")
fig.show()

* The median for surviving is 26 and the same for not surviving is 10.5

In [ ]:
fig1 = px.histogram(train, x="Fare", nbins = 50)
fig1.show()

* Generally the fare ranges from 0 to 90. 
* Some rare case that includes the price to be as high as 500.
* The distribution is not normal at all.

In [ ]:
train["Log_Fare"] = np.log(train["Fare"])
fig1 = px.histogram(train, x="Log_Fare", nbins = 50)
fig1.show()

In [ ]:
def filter_train(train,feature,value):
    return train[train[feature]==value]

In [ ]:
survived = filter_train(train,"Survived",1)
not_survived = filter_train(train,"Survived",0)
fig1 = px.histogram(survived, x="Fare", nbins = 50, template="plotly_dark", barmode = 'overlay', color ='Survived')
fig2 = px.histogram(not_survived, x="Fare", nbins = 50, template="ggplot2", barmode = 'overlay', color ='Survived')
fig1.update_layout(title = 'Histogram of Customers’ Fare',xaxis_title = "Fare", yaxis_title = "Standard Count", width = 800, height = 600)
fig1.add_trace(fig2.data[0])
fig1.show()

* clear indication that most of the passenger who didn't survived are paid less for fare
* there is an overlay of fare price but the trend is clear 
* take away: if you pay more your chance of survival increases [logical]

In [ ]:

Pclass1 = filter_train(train,"Pclass",1)
Pclass2 = filter_train(train,"Pclass",2)
Pclass3 = filter_train(train,"Pclass",3)
fig1 = px.histogram(Pclass1, x="Fare", nbins = 50, template="plotly_dark", barmode = 'overlay', color ='Pclass',color_discrete_sequence=px.colors.qualitative.G10)
fig2 = px.histogram(Pclass2, x="Fare", nbins = 50, template="ggplot2", barmode = 'overlay', color ='Pclass')
fig3 = px.histogram(Pclass3, x="Fare", nbins = 50, template="ggplot2", barmode = 'overlay', color ='Pclass')
fig1.update_layout(title = 'Histogram of Customers’ Fare with class',xaxis_title = "Fare", yaxis_title = "Standard Count", width = 800, height = 600)
fig1.add_trace(fig2.data[0])
fig1.add_trace(fig3.data[0])
fig1.show()

* the mode of fare for class 1 is around 80 and the price of fare if exceed 100 then it is definitely a class 1 ticket.
* For class 2 the mode of price remains around 20 and it max goes upto 80.
* For class 3 it remains upto 10 the mode.

In [ ]:
train_stack=train.groupby(['Survived','Pclass']).size().reset_index()
train_stack['Percentage']=train.groupby(['Survived','Pclass']).size().groupby(level=0).apply(lambda x:100 * x/float(x.sum())).values
        
train_stack.columns= ['Survived','Pclass', 'Counts','Percentage']
      
train_stack['Percentage'] =   train_stack['Percentage'].map('{:,.2f}%'.format) 
      

In [ ]:
fig = px.bar(train_stack, x = 'Pclass', y = 'Counts', color = 
    'Survived', barmode = 'stack')
fig.update_layout(title = "Survived statistics per class",
     xaxis_title = 'Survived', yaxis_title = 'Counts', 
     width = 800, height = 600)
fig.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(x='Pclass',y='Fare',data=train, palette='rainbow',hue="Survived")
plt.title("Fare by Passenger Class, Titanic")

### Analysis of Fare and Age

In [ ]:
sns.distplot(train["Age"], color="r")

In [ ]:
plt.figure(figsize=(10,8))
sns.displot(train,x="Age",hue="Survived")

In [ ]:
g = sns.FacetGrid(train, col="Pclass",  row="Survived")
g.map_dataframe(sns.histplot, x="Age")

* Any insight can you spot from these plots
* It is not clearly visible if these plots have any significance over model building 
* one  common interpretation is that for all the class most survived are the one for whom age remains between 20 to 40

In [ ]:
g = sns.FacetGrid(train, col="Pclass",  row="Survived",hue="Sex")
g.map_dataframe(sns.histplot, x="Age")
g.add_legend()

* for most of the age range in pclass if sex is female she is more likely to survived
* Same for pclass 1 and class 3 aswell 
* The dead female count for pclass1 and 2 is negligble so rich woman survives more than man.

In [ ]:
g = sns.FacetGrid(train, col="Pclass",row="Survived")
g.map_dataframe(sns.scatterplot, x="Age", y="Fare", hue="Sex")
g.add_legend()

* Nothing is clear from this plot.

In [ ]:
from fitter import Fitter, get_common_distributions, get_distributions


In [ ]:
Age = train["Age"].dropna().values

In [ ]:
f = Fitter(Age,
           distributions=['gamma',
                          'lognorm',
                          "beta",
                          "burr",
                          "norm"])
f.fit()
f.summary()

In [ ]:
Fare = train.Fare.values
f = Fitter(Fare,
           distributions=['gamma',
                          'lognorm',
                          "beta",
                          "burr",
                          "norm"])
f.fit()
f.summary()

### Bin The Age

In [ ]:
bins = [0, 20,30, 40,55,90]
labels =[1,2,3,4,5]
train['binned_age'] = pd.cut(train['Age'], bins,labels=labels)


In [ ]:
sns.countplot(train["binned_age"])

In [ ]:
g = sns.FacetGrid(train, col="Pclass",  row="Survived",hue="Sex")
g.map_dataframe(sns.countplot, x="binned_age")
g.add_legend()

* Wow what a plot just validated our hypothesis .
* If you are a female chances of your survival will gonna increases a lot no matter of your age.

### is more age means more money?

In [ ]:
ax = sns.boxplot(x="binned_age", y="Fare", data=train)

* Well our hypothesis is more or less true and we can identify that the most expensive ticket belongs to a person aged between 30-40.

### Is this same across male and female and Pclass?

In [ ]:
g = sns.FacetGrid(train, col="Pclass",row="Survived")
g.map_dataframe(sns.boxplot, x="binned_age", y="Fare", hue="Sex")
g.add_legend()

* for class 1  the female used to have more fare for tickets and they tend to survived 
* for other classes hard to figure out.

### Embarked Analysis

In [ ]:
sns.countplot(train["Embarked"])

In [ ]:
g = sns.FacetGrid(train, col="Pclass",row="Survived")
g.map_dataframe(sns.countplot, x="Embarked", hue="Sex")
g.add_legend()

* for class 1 and 2 the most of the surviving female are with Embarked C 
* for class 3 it is male who survived more and from Embarked S but with less difference.

In [ ]:
train_stack=train.groupby(['Survived','Embarked']).size().reset_index()
train_stack['Percentage']=train.groupby(['Survived','Embarked']).size().groupby(level=0).apply(lambda x:100 * x/float(x.sum())).values
        
train_stack.columns= ['Survived','Embarked', 'Counts','Percentage']
      
train_stack['Percentage'] =   train_stack['Percentage'].map('{:,.2f}%'.format) 

In [ ]:
fig = px.bar(train_stack, x = 'Embarked', y = 'Counts', color = 
    'Survived', barmode = 'stack')
fig.update_layout(title = "Survived statistics per Embarked",
     xaxis_title = 'Survived', yaxis_title = 'Counts', 
     width = 800, height = 600)
fig.show()

* Interesting fact is that the survival  in Embarked C is more then death.
* Let's make one hypothesis  and test it : Embarkment C has fare compatively higher than other ones.

In [ ]:
sns.catplot(x="Embarked", y="Fare", hue="Survived", data=train)

* something interesting is that Emabrked c has the fare of 500+ but rather data point doesn't make it possible to validate our claim and we will be required statistical test to validate our thought.

### Sibling

In [ ]:
sns.countplot(train["SibSp"])

* Analysis of the SibSp is meaningless in these way 
* We will make column where the person is either alone or not.

In [ ]:
train.loc[train["SibSp"]>0,"SibSp"] = 1

In [ ]:
sns.countplot(train["SibSp"])

* better , Now we can run some meaningful analysis with it.

In [ ]:
g = sns.FacetGrid(train, col="Pclass",row="Sex")
g.map_dataframe(sns.countplot, x="SibSp", hue="Survived")
g.add_legend()

* It is hard to find any pattern from the graph.

In [ ]:
train.head()

In [ ]:
train.drop(["PassengerId","Name","Age","Parch","Ticket","Cabin","Log_Fare"],axis=1,inplace=True)

In [ ]:
train

In [ ]:
sex = pd.get_dummies(train["Sex"],drop_first=True).rename(columns=lambda x:'Sex_' +str(x))
pclass = pd.get_dummies(train["Pclass"],drop_first=True).rename(columns=lambda x:'Pclass_' +str(x))
embarked = pd.get_dummies(train["Embarked"],drop_first=True).rename(columns=lambda x:'Embarked_' +str(x))
sibsp = pd.get_dummies(train["SibSp"],drop_first=True).rename(columns=lambda x:'SibSp_' +str(x))
age = pd.get_dummies(train["binned_age"],drop_first=True).rename(columns=lambda x:'Age_' +str(x))

In [ ]:
train = pd.concat([train,sex,pclass,embarked,sibsp,age],axis=1)

In [ ]:
train.head()

In [ ]:
train.drop(["Pclass","Sex","SibSp","Embarked","binned_age"],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train["Sex_male"] =  train["Sex_male"].astype(np.float64)
train["Pclass_2"] =  train["Pclass_2"].astype(np.float64)
train["Pclass_3"] =  train["Pclass_3"].astype(np.float64)
train["Embarked_Q"] =  train["Embarked_Q"].astype(np.float64)
train["Embarked_S"] =  train["Embarked_S"].astype(np.float64)
train["SibSp_1"] =  train["SibSp_1"].astype(np.float64)
train["Age_2"] = train["Age_2"].astype(np.float64)
train["Age_3"] =  train["Age_3"].astype(np.float64)
train["Age_4"] =  train["Age_4"].astype(np.float64)
train["Age_5"] =  train["Age_5"].astype(np.float64)

* Our data cleaning is done next step Model Building.

## Model

## Logistic Regression

In [ ]:
X = train.drop('Survived',axis = 1)
y = train["Survived"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4,test_size=0.2)

In [ ]:
logmodel = LogisticRegression(solver='liblinear')

In [ ]:
logmodel.fit(x_train,y_train)

In [ ]:
predictions = logmodel.predict(x_test)

In [ ]:
print(metrics.classification_report(y_test,predictions))

In [ ]:
print(metrics.confusion_matrix(y_test,predictions))

# Test Data 

In [ ]:
test.head()

In [ ]:
test['Age'].fillna(value=median, inplace=True)

In [ ]:
test["Embarked"].fillna(value="Q", inplace=True)

In [ ]:
test.info()

In [ ]:
test['binned_age'] = pd.cut(test['Age'], bins,labels=labels)

In [ ]:
test.loc[test["SibSp"]>0,"SibSp"] = 1

In [ ]:
test.drop(["PassengerId","Name","Age","Parch","Ticket","Cabin"],axis=1,inplace=True)

In [ ]:
sex = pd.get_dummies(test["Sex"],drop_first=True).rename(columns=lambda x:'Sex_' +str(x))
pclass = pd.get_dummies(test["Pclass"],drop_first=True).rename(columns=lambda x:'Pclass_' +str(x))
embarked = pd.get_dummies(test["Embarked"],drop_first=True).rename(columns=lambda x:'Embarked_' +str(x))
sibsp = pd.get_dummies(test["SibSp"],drop_first=True).rename(columns=lambda x:'SibSp_' +str(x))
age = pd.get_dummies(test["binned_age"],drop_first=True).rename(columns=lambda x:'Age_' +str(x))

In [ ]:
test = pd.concat([test,sex,pclass,embarked,sibsp,age],axis=1)

In [ ]:
test.drop(["Pclass","Sex","SibSp","Embarked","binned_age"],axis=1,inplace=True)

In [ ]:
test["Sex_male"] =  test["Sex_male"].astype(np.float64)
test["Pclass_2"] =  test["Pclass_2"].astype(np.float64)
test["Pclass_3"] =  test["Pclass_3"].astype(np.float64)
test["Embarked_Q"] =  test["Embarked_Q"].astype(np.float64)
test["Embarked_S"] =  test["Embarked_S"].astype(np.float64)
test["SibSp_1"] =  test["SibSp_1"].astype(np.float64)
test["Age_2"] = test["Age_2"].astype(np.float64)
test["Age_3"] =  test["Age_3"].astype(np.float64)
test["Age_4"] =  test["Age_4"].astype(np.float64)
test["Age_5"] =  test["Age_5"].astype(np.float64)

In [ ]:
test["Fare"].fillna(0,inplace=True)

In [ ]:
predict_test_y =  logmodel.predict(test)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predict_test_y
    })
submission.to_csv('submission.csv', index=False)

## score: 0.76076

## KNN 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)



In [ ]:
model.fit(x_train,y_train)

In [ ]:
predicted= model.predict(test)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission1.csv', index=False)

## score: 0.69138

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## Score 0.69617

## Naive-Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## score:0.72248

In [ ]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## score: 0.76315 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## score: 0.74401

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## score: 0.75119

In [ ]:
from sklearn import svm

model = svm.SVC(kernel='linear')
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## score 0.76555

In [ ]:
from sklearn import svm

model = svm.SVC(kernel='rbf')
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## score 0.66028

In [ ]:
from xgboost import XGBClassifier


model = XGBClassifier()
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## score: 0.76315

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
model.fit(X,y)
predicted= model.predict(test)
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Survived": predicted
    })
submission.to_csv('submission2.csv', index=False)

## score: 0.74641

## Next step refactor code , explain algorithm, add big data solution, add tensorflow code